In [1]:
from google.cloud import bigquery

## Parametros

In [2]:
client_bq = bigquery.Client()

In [1]:
dataset = 'co-keralty-costomedico.Diabetes_avicena_survival'
table_avicena = 'prj-dev-repository-col.bq_finalzone_avicena.consulta_externa'

table_pacientes = f'{dataset}.pacientes'
main_table = f'{dataset}.diabetes'

filter_3_years = "DATE({}) >= DATE(DATE_FROM_UNIX_DATE(UNIX_DATE(CURRENT_DATE()) - 1096))"
filter_3_years_1_month = "DATE({}) <= DATE(DATE_FROM_UNIX_DATE(UNIX_DATE(CURRENT_DATE()) - 1065))"
filter_2_years_11_month = "DATE({}) > DATE(DATE_FROM_UNIX_DATE(UNIX_DATE(CURRENT_DATE()) - 1065))"
filter_pacientes = f"""SELECT DISTINCT numero_identificacion_paciente FROM `{table_pacientes}`"""

dx_cardio = ['I630','I631','I632','I633','I634','I635','I638','I639','I653','I658','I659','I660','I661','I662','I664','I668',
            'I669','I672','I693','I698','I500','I509','I518','Y840','I210','I211','I212','I213','I214','I219','I220','I221',
            'I228','I229','I230','I231','I232','I233','I234','I235','I238','I240','I250','I251','I252','I702','I708','I709',
            'I739','I236','I240','I513','I81X','J960','J961','J969','P285','E271','E273','E274','E283','E310','E894','I051',
            'I052','I061','I062','I071','I110','I119','I120','I129','I130','I131','I132','I340','I351','I361','I371','I500',
            'I501','I509','I872','J951','J952','J953','J960','J961','J969','K704','K720','K721','K729','N170','N171','N172',
            'N178','N179','N180','N188','N189','N19X','N990','O084','O904','P285','P290','P960','Q222','Q231','Q233','Z353',
            'Z597','I210','I211','I212','I213','I214','I219','I220','I221','I228','I229','I230','I231','I232','I252','I256',
            'P294','I200','I201','I208','I209','I10X','I110','I119','I120','I129','I130','I131','I132','I139','I150','I151']

med_hipertension = ['C02AB0113C01','C02AB0113C02','C02AC0113C01','C02CA0113C01','C03EA0113C02',
                                     'C03EA0113C03','C07AA0513C01','C07AA0513C02','C07AB0213C02','C07AB0213C03',
                                     'C07AB0214C01','C07AB0214C02','C07AB0214C03','C07AB0214C04','C07AB0231C01',
                                     'C07AB0313C01','C07AB0313C02','C07AB0713C01','C07AB0713C07','C07CB0313C01',
                                     'C08CA0101C01','C08CA0101C02','C08CA0101C08','C08CA0113C01','C08CA0113C02',
                                     'C08CA0113C03','C08CA0113C04','C08CA0113C05','C08CA0113C07','C08CA0113C08',
                                     'C08CA0113C10','C08CA0501C01','C08CA0503C01','C08CA0514C02','C08DA0103C02',
                                     'C08DA0113C01','C08DA0113C03','C08DA0114C02','C08DA0131C01','C08GA0213C01',
                                     'C09AA0113C01','C09AA0113C02','C09AA0213C02','C09AA0213C03','C09BB0413C01',
                                     'C09BB0413C02','C09BB0413C03','C09BB0413C04','C09BB1003C01','C09BB1013C01',
                                     'C09BX0113C02','C09BX0113C03','C09CA0113C02','C09CA0113C03','C09CA0301C14',
                                     'C09CA0813C01','C09CA0813C02','C09CA0813C03','C09DA0113C02','C09DA0113C03',
                                     'C09DA0113C04','C09DA0613C01','C09DB0101C04','C09DB0113C01','C09DB0113C02',
                                     'C09DB0113C05','C09DB0413C02','C09DB0413C03','C09DB0413C05','C09DB0513C01',
                                     'C09DB0513C02','C09DB0513C03','C09DB0513C04','C09DB0601C01','C09DX0113C02',
                                     'C09DX0113C04','C09DX0113C05','MAGISTR6018']

examns_HDL = ['Colesterol HDL','Colesterol HDL (mg/dl)','HDL (mg/dl)']
examns_LDL = ['Colesterol LDL','Colesterol LDL (mg/dl)','LDL (mg/dl)']
examns_trigliceridos = ['Triglicéridos','Triglicéridos (mg/dl)']

column_year = """EXTRACT(YEAR
    FROM
      fecha_apertura_folio) AS year,"""

column_month = """EXTRACT(MONTH
    FROM
      fecha_apertura_folio) AS month,"""

## Consultas

### Tabla pacientes

In [2]:
query_pacientes = f"""

CREATE OR REPLACE TABLE
  `{table_pacientes}` AS
SELECT
  numero_identificacion_paciente,
IF
  (SUM(diabetes) >= 1, 1, 0) AS diabetes
FROM (
  SELECT
    DISTINCT numero_identificacion_paciente,
    ABS(DATE_DIFF(DATE(fecha_apertura_folio), DATE(fecha_nacimiento_paciente), YEAR)) AS edad,
  IF
    (dx_corta IN ('Diabetes'),1,0) AS diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND {filter_3_years_1_month.format('fecha_apertura_folio')}
    AND estado_folio = 'F'
  UNION ALL
  SELECT
    DISTINCT numero_identificacion_paciente,
    ABS(DATE_DIFF(DATE(fecha_apertura_folio), DATE(fecha_nacimiento_paciente), YEAR)) AS edad,
  IF
    (descripcion_corta_diagnostico IN ('Diabetes'),1,0) AS diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.antecedentes`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND {filter_3_years_1_month.format('fecha_apertura_folio')}
    AND estado_folio = 'F' )
WHERE
  diabetes = 0
  AND edad >= 18
GROUP BY
  1

"""

In [3]:
print(query_pacientes)



CREATE OR REPLACE TABLE
  `co-keralty-costomedico.Diabetes_avicena_survival.pacientes` AS
SELECT
  numero_identificacion_paciente,
IF
  (SUM(diabetes) >= 1, 1, 0) AS diabetes
FROM (
  SELECT
    DISTINCT numero_identificacion_paciente,
    ABS(DATE_DIFF(DATE(fecha_apertura_folio), DATE(fecha_nacimiento_paciente), YEAR)) AS edad,
  IF
    (dx_corta IN ('Diabetes'),1,0) AS diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`
  WHERE
    DATE(fecha_apertura_folio) >= DATE(DATE_FROM_UNIX_DATE(UNIX_DATE(CURRENT_DATE()) - 1096))
    AND DATE(fecha_apertura_folio) <= DATE(DATE_FROM_UNIX_DATE(UNIX_DATE(CURRENT_DATE()) - 1065))
    AND estado_folio = 'F'
  UNION ALL
  SELECT
    DISTINCT numero_identificacion_paciente,
    ABS(DATE_DIFF(DATE(fecha_apertura_folio), DATE(fecha_nacimiento_paciente), YEAR)) AS edad,
  IF
    (descripcion_corta_diagnostico IN ('Diabetes'),1,0) AS diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.antecedentes`
  WHERE


In [5]:
client_bq.query(query_pacientes).result()

## Tabla principal

In [6]:
query_avicena = f"""

SELECT
  numero_identificacion_paciente,
  year,
  month,
  MAX(edad) AS edad,
  --AVG(tension_arterial_sistole) AS tension_arterial_sistole,
  --AVG(tension_arterial_diastole) AS tension_arterial_diastole,
  AVG(peso) AS peso,
  AVG(talla) AS talla,
  MAX(genero_paciente) AS genero_paciente,
  MAX(raza_paciente) AS raza_paciente,
  MAX(nivel_academico_paciente) AS nivel_academico_paciente,
FROM (
  SELECT
    principal.numero_identificacion_paciente,
    {column_year.format('fecha_apertura_folio')}
    {column_month.format('fecha_apertura_folio')}
    fecha_nacimiento_paciente,
    DATE_DIFF(DATE(fecha_apertura_folio), DATE(fecha_nacimiento_paciente), YEAR) as edad,
    --tension_arterial_sistole,
    --tension_arterial_diastole,
    peso,
    talla,
    genero_paciente,
    raza_paciente,
    nivel_academico_paciente,
  FROM
    `{table_avicena}` AS principal
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F'
    --AND tension_arterial_sistole IS NOT NULL
    --AND tension_arterial_diastole IS NOT NULL
    --AND peso IS NOT NULL
    --AND talla IS NOT NULL
    --AND genero_paciente IS NOT NULL
    --AND raza_paciente IS NOT NULL
    --AND nivel_academico_paciente IS NOT NULL
    AND numero_identificacion_paciente IN ({filter_pacientes}))
GROUP BY
  numero_identificacion_paciente,
  year,
  month

"""

query_ant_cardio = f"""

  SELECT numero_identificacion_paciente, year, month, MAX(ant_cardio) as ant_cardio FROM (SELECT
    numero_identificacion_paciente,
    {column_year}
    {column_month}
    IF(codigo_dx IN {tuple(dx_cardio)},1,0) as ant_cardio
  FROM
    `{table_avicena}`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F' )
    
  GROUP BY 1,2,3

"""

query_HDL = f"""

  SELECT 
    numero_identificacion_paciente,
    year, 
    month,
    AVG(HDL) AS HDL,
  
  FROM (SELECT 
    numero_identificacion_paciente,
    {column_year}
    {column_month}
    IF(examen_laboratorio IN {tuple(examns_HDL)},SAFE_CAST(resultado_examen_laboratorio AS FLOAT64),-999) AS HDL,
  FROM
    `{table_avicena}`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F'
    )
  WHERE 
    HDL != -999
  GROUP BY
    1,2,3


"""

query_LDL = f"""

SELECT 
    numero_identificacion_paciente,
    year, 
    month,
    AVG(LDL) AS LDL 
  
  FROM (SELECT 
    numero_identificacion_paciente,
    {column_year}
    {column_month}
    IF(examen_laboratorio IN {tuple(examns_LDL)},SAFE_CAST(resultado_examen_laboratorio AS FLOAT64),-999) AS LDL,
  FROM
    `{table_avicena}`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F'
    )
  WHERE 
    LDL != -999
  GROUP BY
    1,2,3

"""

query_trigliceridos = f"""

  SELECT 
    numero_identificacion_paciente,
    year, 
    month,
    AVG(trigliceridos) AS trigliceridos 
  
  FROM (SELECT 
    numero_identificacion_paciente,
    {column_year}
    {column_month}
    IF(examen_laboratorio IN {tuple(examns_trigliceridos)},SAFE_CAST(resultado_examen_laboratorio AS FLOAT64),-999) AS trigliceridos,
  FROM
    `{table_avicena}`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F'
    )
  WHERE 
    trigliceridos != -999
  GROUP BY
    1,2,3

"""

query_perimetro_abdominal = f"""

  SELECT numero_identificacion_paciente, year, month, AVG(PERIMETRO_ABDOMINAL) AS perimetro_abdominal FROM (SELECT
    DISTINCT main.numero_identificacion_paciente,
    {column_year}
    {column_month}
    perimetro.PERIMETRO_ABDOMINAL
  FROM
    `{table_avicena}` AS main

  INNER JOIN (
                              SELECT
                                DISTINCT FOLIO,
                                PERIMETRO_ABDOMINAL
                              FROM
                                `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_EXAMEN_FISICO`
                              WHERE
                                PERIMETRO_ABDOMINAL IS NOT NULL) AS perimetro
                            ON
                              main.folio = perimetro.FOLIO

  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F')

  GROUP BY 
    1,2,3

"""

query_ant_fam_dm = f"""

  SELECT
    numero_identificacion_paciente,
    year,
    month,
  IF
    (SUM(ant_familiar_dm) >= 1, 1, 0) AS ant_familiar_dm
  FROM (
    SELECT
      DISTINCT numero_identificacion_paciente,
      {column_year}
      {column_month}
    IF
      (dx_corta IN ('Antecedente familiar de diabetes'),1,0) AS ant_familiar_dm,
    FROM
      `{table_avicena}`
    WHERE
      {filter_3_years.format('fecha_apertura_folio')}
      AND estado_folio = 'F'
    UNION ALL
    SELECT
      DISTINCT numero_identificacion_paciente,
      {column_year}
      {column_month}
    IF
      (descripcion_corta_diagnostico IN ('Antecedente familiar de diabetes'),1,0) AS ant_familiar_dm,
    FROM
      `prj-dev-repository-col.bq_finalzone_avicena.antecedentes`
    WHERE
      {filter_3_years.format('fecha_apertura_folio')}
      AND estado_folio = 'F' )
  GROUP BY
    1,2,3

"""


query_med_hipertension = f"""

  SELECT numero_identificacion_paciente, year, month, MAX(med_hipertension) as med_hipertension FROM (SELECT 
    numero_identificacion_paciente,
    {column_year}
    {column_month}
    IF(codigo_atc IN {tuple(med_hipertension)},1,0) as med_hipertension
  FROM
    `{table_avicena}`
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND estado_folio = 'F' )
  GROUP BY
    1,2,3

"""

query_ejercicio_diario = f"""

SELECT
  numero_identificacion_paciente,
  year,
  month,
  hace_ejercicio
FROM (
  SELECT
    DISTINCT pregunta.RESPUESTA_ANTECEDENTE,
    {column_year}
    {column_month}
    respuesta.DESCRIPCION_ALTERNA AS hace_ejercicio,
    consulta.numero_identificacion_paciente,
    ROW_NUMBER() OVER (PARTITION BY consulta.numero_identificacion_paciente ORDER BY consulta.fecha_apertura_folio DESC) AS orden
  FROM
    `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDEN_ITEM` AS pregunta
  INNER JOIN (
    SELECT
      *
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_ENC_PARAMETRO`) AS respuesta
  ON
    pregunta.PARAM_ITEM_RESPUESTA = respuesta.PARAMETRO
  INNER JOIN (
    SELECT
      RESPUESTA_ANTECEDENTE,
      FECHA_REGISTRO,
      FOLIO
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDENTE`) AS folio
  ON
    folio.RESPUESTA_ANTECEDENTE = pregunta.RESPUESTA_ANTECEDENTE
  LEFT JOIN (
    SELECT
      DISTINCT folio,
      fecha_apertura_folio,
      numero_identificacion_paciente
    FROM
      `{table_avicena}`) AS consulta
  ON
    folio.FOLIO = consulta.folio
  WHERE
    {filter_3_years.format('fecha_apertura_folio')}
    AND pregunta.PREGUNTA = 1441
    AND pregunta.PARAM_ITEM_RESPUESTA IS NOT NULL )
WHERE
  orden = 1

"""

query_diag_diabetes = f"""

SELECT
    numero_identificacion_paciente,
    year,
    month,
  IF
    (SUM(diabetes) >= 1, 1, 0) AS diabetes
  FROM (
    SELECT
      DISTINCT numero_identificacion_paciente,
      {column_year}
      {column_month}
    IF
      (dx_corta IN ('Diabetes'),1,0) AS diabetes,
    FROM
      `{table_avicena}`
    WHERE
      {filter_3_years.format('fecha_apertura_folio')}
      AND estado_folio = 'F'
    UNION ALL
    SELECT
      DISTINCT numero_identificacion_paciente,
      {column_year}
      {column_month}
    IF
      (descripcion_corta_diagnostico IN ('Diabetes'),1,0) AS diabetes,
    FROM
      `prj-dev-repository-col.bq_finalzone_avicena.antecedentes`
    WHERE
      {filter_3_years.format('fecha_apertura_folio')}
      AND estado_folio = 'F' )
  GROUP BY
    1,2,3

"""

In [7]:
main_query = f"""

CREATE OR REPLACE TABLE
  `{main_table}`
AS 

WITH

----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE LA TABLA PRINCIPAL
---------------------------------------------------------------------- 

principal AS (
  {query_avicena}
),

----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE ANTECEDENTES CARDIOVASCULARES
---------------------------------------------------------------------- 
  antecedentes_cardiovasculares AS (
    {query_ant_cardio}
  ),

----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE CONSUMO DE MEDICAMENTOS DE HIPERTENSION
---------------------------------------------------------------------- 

medicamentos_hipertension AS (
  {query_med_hipertension}
),


----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE ANETECEDENTES FAMILIARES CON DIABETES
---------------------------------------------------------------------- 

antecedentes_familiar_dm AS (
  {query_ant_fam_dm}
),

----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE EXAMENES HDL
---------------------------------------------------------------------- 

examenes_HDL AS (
  {query_HDL}
),


----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE EXAMENES LDL
---------------------------------------------------------------------- 

examenes_LDL AS (
  {query_LDL}
),


----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE EXAMENES TRIGLICERIDOS
---------------------------------------------------------------------- 

examenes_TRIGLICERIDOS AS (
  {query_trigliceridos}
),


----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE PERIMETROS ABDOMINALES
---------------------------------------------------------------------- 

tabla_perimetros AS (
  {query_perimetro_abdominal}
),


----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE EJERCICIO DIARIO
---------------------------------------------------------------------- 

tabla_ejercicio AS (
  {query_ejercicio_diario}
),

----------------------------------------------------------------------
-- CONSULTA PARA LA CREACION DE DIAGNOSTICO DE DIABETES
---------------------------------------------------------------------- 

tabla_diabetes_diagnostico AS (
  {query_diag_diabetes}
)






SELECT
  DISTINCT

  principal.numero_identificacion_paciente,
  principal.year,
  principal.month,

  -- Variables Numericas
  principal.edad,
  principal.peso,
  principal.talla,
  examenes_HDL.HDL,
  examenes_LDL.LDL,
  examenes_TRIGLICERIDOS.trigliceridos,
  tabla_perimetros.perimetro_abdominal,

  -- Variables Categoricas
  principal.genero_paciente,
  principal.raza_paciente,
  principal.nivel_academico_paciente,
  antecedentes_cardiovasculares.ant_cardio,
  medicamentos_hipertension.med_hipertension,
  antecedentes_familiar_dm.ant_familiar_dm,
  tabla_ejercicio.hace_ejercicio,

  -- Variable Objetivo
  tabla_diabetes_diagnostico.diabetes
FROM
  principal
LEFT JOIN
  antecedentes_cardiovasculares ON antecedentes_cardiovasculares.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND antecedentes_cardiovasculares.year = principal.year
  AND antecedentes_cardiovasculares.month = principal.month
LEFT JOIN
  antecedentes_familiar_dm ON antecedentes_familiar_dm.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND antecedentes_familiar_dm.year = principal.year
  AND antecedentes_familiar_dm.month = principal.month
LEFT JOIN
  tabla_ejercicio ON tabla_ejercicio.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND tabla_ejercicio.year = principal.year
  AND tabla_ejercicio.month = principal.month
LEFT JOIN
  medicamentos_hipertension ON medicamentos_hipertension.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND medicamentos_hipertension.year = principal.year
  AND medicamentos_hipertension.month = principal.month
LEFT JOIN
  examenes_HDL ON examenes_HDL.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND examenes_HDL.year = principal.year
  AND examenes_HDL.month = principal.month
LEFT JOIN
  examenes_LDL ON examenes_LDL.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND examenes_LDL.year = principal.year
  AND examenes_LDL.month = principal.month
LEFT JOIN
  examenes_TRIGLICERIDOS ON examenes_TRIGLICERIDOS.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND examenes_TRIGLICERIDOS.year = principal.year
  AND examenes_TRIGLICERIDOS.month = principal.month
LEFT JOIN
  tabla_perimetros ON tabla_perimetros.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND tabla_perimetros.year = principal.year
  AND tabla_perimetros.month = principal.month
LEFT JOIN
  tabla_diabetes_diagnostico ON tabla_diabetes_diagnostico.numero_identificacion_paciente = principal.numero_identificacion_paciente 
  AND tabla_diabetes_diagnostico.year = principal.year
  AND tabla_diabetes_diagnostico.month = principal.month
WHERE 
  principal.edad >= 18



"""

In [8]:
client_bq.query(main_query).result()

In [9]:
query_fechas = """

CREATE OR REPLACE TABLE
  `co-keralty-costomedico.Diabetes_avicena_survival.fechas` AS
WITH
  si_diabetes AS (
  SELECT
    DISTINCT CAST(numero_identificacion_paciente AS STRING) AS numero_identificacion_paciente,
    MIN(DATE(CONCAT(year,'-',month,'-01'))) AS fecha_con
  FROM
    `co-keralty-costomedico.Diabetes_avicena_survival.diabetes` AS fecha_si_diabetes
  WHERE
    diabetes = 1
  GROUP BY
    numero_identificacion_paciente),
  no_diabetes AS (
  SELECT
    DISTINCT CAST(numero_identificacion_paciente AS STRING) AS numero_identificacion_paciente,
    MAX(DATE(CONCAT(year,'-',month,'-01'))) AS fecha_sin,
  FROM
    `co-keralty-costomedico.Diabetes_avicena_survival.diabetes`
  WHERE
    diabetes = 0
  GROUP BY
    numero_identificacion_paciente)


    
SELECT
  DISTINCT numero_identificacion_paciente,
IF
  (fecha_con <= fecha_sin
    AND fecha_sin != DATE(CURRENT_DATE()), fecha_sin, fecha_con) AS fecha
FROM (
  SELECT
    no_diabetes.numero_identificacion_paciente,
    no_diabetes.fecha_sin,
  IF
    (si_diabetes.fecha_con IS NULL, DATE(CURRENT_DATE()), si_diabetes.fecha_con) AS fecha_con
  FROM
    no_diabetes
  LEFT JOIN
    si_diabetes
  ON
    CAST(no_diabetes.numero_identificacion_paciente AS STRING) = CAST(si_diabetes.numero_identificacion_paciente AS STRING) )


"""

: 